In [1]:
from drn_interactions.interactions.loaders import SpontaneousActivityLoader, StateInteractionsLoader
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.pairwise import PairwiseCorr
from drn_interactions.interactions.graph_clustering import df_to_graph
from drn_interactions.config import Config, ExperimentInfo
import numpy as np
from drn_interactions.interactions.graph import GraphAttributes, NodeAttributes
import pandas as pd
import seaborn as sns
from drn_interactions.io import load_derived_generic, load_distances
from drn_interactions.transforms.graph import GraphTransformer

### Get Graph Results

- Graph DF (SWP, Avg Clutering etc)
- Node (Centrality, NT, Clustering etc)
- Edge DF (Weight, Distance, NT Combo, Same NT)

In [2]:
def load_responders():
    slow_responders_shock = load_derived_generic(
        "slow_ts_foot_shock_unit_responders_pre_to_shock.csv"
    )[["neuron_id", "diff_inv", "sig"]].assign(
        response_fs_slow=lambda x: np.where(
            x["sig"] == False,
            "no_response",
            np.where(x["diff_inv"] < 0, "inhibited", "activated"),
        )
    )[["neuron_id", "response_fs_slow"]]

    fast_responders = load_derived_generic("fast_fs_foot_shock_unit_responders.csv")[
        ["neuron_id", "Diff", "sig"]
    ].assign(
        response_fs_fast=lambda x: np.where(
            x["sig"] == False,
            "no_response",
            np.where(x["Diff"] < 0, "inhibited", "activated"),
        )
    )[["neuron_id", "response_fs_fast"]]

    bs_response = load_derived_generic("brain_states_spikerate_responders.csv")[
        ["neuron_id", "Diff", "sig"]
    ].assign(
        response_bs=lambda x: np.where(
            x["sig"] == False,
            "no_response",
            np.where(x["Diff"] < 0, "inhibited", "activated"),
        )
    )[["neuron_id", "response_bs"]]

    df_responders = slow_responders_shock.merge(fast_responders, how="outer").merge(bs_response, how="outer")
    return df_responders
    

In [3]:

df_ensembles = load_derived_generic("ensembles/spont - ensembles - true.csv")
df_ensembles["in_ensemble"] = np.where(df_ensembles["ensemble_id"] == -1, False, True)

graph_attrs = GraphAttributes(inverse_distance=True)
node_attrs = NodeAttributes()
df_responders = load_responders()
neuron_types = load_derived_generic("neuron_types.csv")[
    ["neuron_id", "neuron_type", "session_name"]
]
df_distance = load_distances()
sessions = neuron_types["session_name"].dropna().unique()

g_transform = GraphTransformer(
    relabel_nodes=True,
    weight_attr="weight",
    neuron_types=neuron_types,
    df_distance=df_distance,
    df_ensemble=df_ensembles,
)
graph_dfs = []
node_dfs = []
node_mappers = []
edge_dfs = []

for shuffle_higher in (False, True):
    for session in sessions:
        loader = SpontaneousActivityLoader(
            session_name=session, bin_width=1, block="pre", t_start=0, t_stop=1800
        )
        preprocessor = InteractionsPreprocessor()
        pairwise = PairwiseCorr(rectify=True, shuffle=shuffle_higher)

        spikes = preprocessor(loader())
        df_affinity = (
            pairwise.fit(spikes)
            .get_adjacency_df()
            .dropna(axis=1, thresh=5)
            .dropna(axis=0, thresh=5)
        )
        G = df_to_graph(df_affinity, rename_nodes=True)
        graph_stats = graph_attrs.get_graph_attributes(G).assign(session=session, shuffle=shuffle_higher)
        node_stats = node_attrs.get_node_attributes(G, node_name="neuron_id").assign(
            session=session, shuffle=shuffle_higher
        )
        edge_stats =g_transform.graph_to_edge_df(G).assign(session=session, shuffle=shuffle_higher)

        graph_dfs.append(graph_stats)
        node_dfs.append(node_stats)
        edge_dfs.append(edge_stats)

df_graph = pd.concat(graph_dfs).reset_index(drop=True)
df_node = pd.concat(node_dfs).reset_index(drop=True).merge(neuron_types[["neuron_id", "neuron_type"]], how="left")
df_node = df_node.merge(df_ensembles[["neuron_id", "in_ensemble"]], how="left")
df_edge = pd.concat(edge_dfs).reset_index(drop=True)

### Compare Graph Properties

In [5]:
import pingouin as pg
from drn_interactions.stats import mannwhitneyu_plusplus
from scipy.stats import wilcoxon, ttest_rel

to_compare = ["avg_deg", "avg_clust", "swp", "avg_path_len"]

def compare_one(df, to_compare, session="session_name", shuffle_col="shuffle"):
    dfp = df.pivot(index=session, columns=shuffle_col, values=to_compare)
    return mannwhitneyu_plusplus(x=dfp.iloc[:, 0], y=dfp.iloc[:, 1], names=["False", "True"], compare_f=wilcoxon)


sers = []
for col in to_compare:
    res = compare_one(df_graph, to_compare=col, session="session", shuffle_col="shuffle")
    res.name = col
    sers.append(res.to_frame())

res_graph = pd.concat(sers, axis=1).T.round(2)
display(res_graph)

/opt/miniconda3/envs/drn/lib/python3.8/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,n_False,n_True,Mean_False,Mean_True,Diff,U,p
avg_deg,22.0,22.0,4.09,4.08,-0.01,125.0,0.97
avg_clust,22.0,22.0,0.23,0.16,-0.06,0.0,0.00
swp,22.0,22.0,0.39,0.40,0.01,85.0,0.69
avg_path_len,22.0,22.0,35.35,5.50,-29.85,3.0,0.00


### Compare Node Properties

In [6]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, KFold


df = df_node.query(
    "shuffle == False"
)

def get_Xy(df, y_col, x_cols=("neuron_type"), shuffle=False):
    X = df[x_cols]
    y = df[y_col].values
    if shuffle:
        y = np.random.permutation(y)
    return X, y


X_str, y_str = get_Xy(df.copy(), y_col="in_ensemble", shuffle=False)
y = LabelEncoder().fit_transform(y_str)
X = OneHotEncoder(sparse=False).fit_transform(X_str.values.reshape(-1, 1))
clf = DecisionTreeClassifier(max_depth=10, criterion="entropy")

score_true = cross_val_score(
    clf, 
    X, 
    y, 
    cv=KFold(shuffle=True), 
    scoring="f1_macro",).mean()

scores_obs = []
for i in range(500):
    X_str, y_str = get_Xy(df.copy(), y_col="in_ensemble", shuffle=True)
    y = LabelEncoder().fit_transform(y_str)
    X = OneHotEncoder(sparse=False).fit_transform(X_str.values.reshape(-1, 1))
    score = cross_val_score(
        clf, 
        X, 
        y, 
        cv=KFold(shuffle=True), 
        scoring="f1_macro",).mean()
    scores_obs.append(score)

scores_boot = np.array(scores_obs)
np.mean(scores_boot >= score_true)

0.104

In [7]:
df = df_node.query(
    "shuffle == False"
)

def get_Xy(df, y_col, x_cols=("neuron_type"), shuffle=False):
    X = df[x_cols]
    y = df[y_col].values
    if shuffle:
        y = np.random.permutation(y)
    return X, y


X_str, y = get_Xy(df.copy(), y_col="clust", shuffle=False)

X = OneHotEncoder(sparse=False).fit_transform(X_str.values.reshape(-1, 1))
clf = DecisionTreeRegressor(max_depth=10)

score_true = cross_val_score(
    clf, 
    X, 
    y, 
    cv=KFold(shuffle=True), 
    scoring="r2",).mean()

scores_obs = []
for i in range(500):
    X_str, y = get_Xy(df.copy(), y_col="clust", shuffle=True)
    X = OneHotEncoder(sparse=False).fit_transform(X_str.values.reshape(-1, 1))
    clf = DecisionTreeRegressor(max_depth=10)
    score = cross_val_score(
        clf, 
        X, 
        y, 
        cv=KFold(shuffle=True), 
        scoring="r2",).mean()
    scores_obs.append(score)

scores_boot = np.array(scores_obs)
np.mean(scores_boot >= score_true)

0.0

In [8]:
df = df_node.query(
    "shuffle == False"
)

def get_Xy(df, y_col, x_cols=("neuron_type"), shuffle=False):
    X = df[x_cols]
    y = df[y_col].values
    if shuffle:
        y = np.random.permutation(y)
    return X, y


X_str, y = get_Xy(df.copy(), y_col="degree", shuffle=False)

X = OneHotEncoder(sparse=False).fit_transform(X_str.values.reshape(-1, 1))
clf = DecisionTreeRegressor(max_depth=10)

score_true = cross_val_score(
    clf, 
    X, 
    y, 
    cv=KFold(shuffle=True), 
    scoring="r2",).mean()

scores_obs = []
for i in range(500):
    X_str, y = get_Xy(df.copy(), y_col="degree", shuffle=True)
    X = OneHotEncoder(sparse=False).fit_transform(X_str.values.reshape(-1, 1))
    clf = DecisionTreeRegressor(max_depth=10)
    score = cross_val_score(
        clf, 
        X, 
        y, 
        cv=KFold(shuffle=True), 
        scoring="r2",).mean()
    scores_obs.append(score)

scores_boot = np.array(scores_obs)
np.mean(scores_boot >= score_true)

0.006

In [9]:
from scipy.stats import chi2_contingency

X = pd.crosstab(df["neuron_type"], df["in_ensemble"])
chi2_contingency(X)

(3.988138427744215,
 0.13614031272622235,
 2,
 array([[ 16.39145907,  77.60854093],
        [ 39.58362989, 187.41637011],
        [ 42.02491103, 198.97508897]]))

In [10]:
from scipy.stats import chi2_contingency

anova = pg.anova(data=df, dv="degree", between="neuron_type")
display(anova)
pg.pairwise_tukey(data=df, dv="degree", between="neuron_type", ).round(2)

,Source,ddof1,ddof2,F,p-unc,np2
0,neuron_type,2,559,11.793685,0.00001,0.040487


,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,FF,SIR,0.19,0.13,0.06,0.01,4.40,0.0,0.54
1,FF,SR,0.19,0.17,0.02,0.01,1.71,0.2,0.21
2,SIR,SR,0.13,0.17,-0.04,0.01,-3.60,0.0,-0.33


In [11]:
anova = pg.anova(data=df, dv="clust", between="neuron_type")
display(anova)
pg.pairwise_tukey(data=df, dv="clust", between="neuron_type", ).round(2)

,Source,ddof1,ddof2,F,p-unc,np2
0,neuron_type,2,559,13.306476,0.000002,0.045445


,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,FF,SIR,0.24,0.18,0.06,0.01,4.52,0.0,0.55
1,FF,SR,0.24,0.23,0.02,0.01,1.48,0.3,0.18
2,SIR,SR,0.18,0.23,-0.04,0.01,-4.04,0.0,-0.37


In [12]:
df

,neuron_id,degree,clust,page_rank,session,shuffle,neuron_type,in_ensemble
0,1,0.150331,0.357448,0.103637,ESHOCK_03_LOC1,False,SIR,False
1,4,0.392565,0.796391,0.217290,ESHOCK_03_LOC1,False,SIR,False
2,5,0.427232,0.416762,0.247035,ESHOCK_03_LOC1,False,FF,False
3,6,0.393381,0.452307,0.218549,ESHOCK_03_LOC1,False,SIR,False
4,8,0.385671,0.785381,0.213489,ESHOCK_03_LOC1,False,SR,False
...,...,...,...,...,...,...,...,...
557,2627,0.066097,0.107281,0.020856,acute_11,False,SIR,False
558,2628,0.030208,0.073201,0.012168,acute_11,False,SIR,True
559,2629,0.072573,0.146359,0.021776,acute_11,False,SIR,False
560,2630,0.043110,0.083524,0.015626,acute_11,False,FF,True


### Compare Edge Properties

In [13]:
# weight ~ distance + nt_comb
from sklearn.compose import ColumnTransformer


df = df_edge.query(
    "shuffle == False"
)

def get_Xy(df, y_col, x_cols=["distance", "nt_comb"], shuffle=False):
    X = df[x_cols]
    y = df[y_col].values
    if shuffle:
        y = np.random.permutation(y)
    return X, y


X_str, y = get_Xy(df.copy(), y_col="weight", shuffle=False)


ct = ColumnTransformer(
    [("ohe", OneHotEncoder(sparse=False), ["nt_comb"])],
    remainder="passthrough",
)
X = ct.fit_transform(X_str)


clf = DecisionTreeRegressor(max_depth=10)

score_true = cross_val_score(
    clf, 
    X, 
    y, 
    cv=KFold(shuffle=True), 
    scoring="r2",).mean()

scores_obs = []
for i in range(500):
    X_str, y = get_Xy(df.copy(), y_col="weight", shuffle=True)
    ct = ColumnTransformer(
        [("ohe", OneHotEncoder(sparse=False), ["nt_comb"])],
        remainder="passthrough",
    )
    X = ct.fit_transform(X_str)
    clf = DecisionTreeRegressor(max_depth=10)
    score = cross_val_score(
        clf, 
        X, 
        y, 
        cv=KFold(shuffle=True), 
        scoring="r2",).mean()
    scores_obs.append(score)

scores_boot = np.array(scores_obs)
np.mean(scores_boot >= score_true)

0.0

In [19]:
from sklearn.compose import ColumnTransformer


df = df_edge.query(
    "shuffle == False"
)

def get_Xy(df, y_col, x_cols=["distance", "nt_comb"], shuffle=False):
    X = df[x_cols]
    y = df[y_col].values
    if shuffle:
        y = np.random.permutation(y)
    return X, y


X_str, y = get_Xy(df.copy(), y_col="same_ensemble", shuffle=False)
y = LabelEncoder().fit_transform(y)

ct = ColumnTransformer(
    [("ohe", OneHotEncoder(sparse=False), ["nt_comb"])],
    remainder="passthrough",
)
X = ct.fit_transform(X_str)

clf = DecisionTreeClassifier(max_depth=10)

score_true = cross_val_score(
    clf, 
    X, 
    y, 
    cv=KFold(shuffle=True), 
    scoring="f1_macro",).mean()

scores_obs = []
for i in range(500):
    X_str, y = get_Xy(df.copy(), y_col="same_ensemble", shuffle=True)
    ct = ColumnTransformer(
        [("ohe", OneHotEncoder(sparse=False), ["nt_comb"])],
        remainder="passthrough",
    )
    y = LabelEncoder().fit_transform(y)
    X = ct.fit_transform(X_str)
    clf = DecisionTreeClassifier(max_depth=10)
    score = cross_val_score(
        clf, 
        X, 
        y, 
        cv=KFold(shuffle=True), 
        scoring="f1_macro",).mean()
    scores_obs.append(score)

scores_boot = np.array(scores_obs)
np.mean(scores_boot >= score_true)

0.0